# Imports

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import os
import librosa
import asyncio
import numpy as np
import pandas as pd
import scipy.stats as stats
import time
import nest_asyncio
from speechpy import feature
import aiofiles
import csv
import math
import parselmouth
import matplotlib.pyplot as plt
import amfm_decompy.pYAAPT as pYAAPT

# Frequency Features

## Extract Features

In [ ]:
async def sleep_sent(count):
    print(f'Time: {time.time() - start_time:.2f} Count Sent: {count}')
    await asyncio.sleep(0.1)

In [ ]:
async def get_frequencies(file):

    file_path = os.path.join(current_path,"CETUC", "Full", file)
    if file[0] == 'F':
        gender = 0
    if file[0] == 'M': 
        gender = 1
    audio_data, sample_rate = await librosa.load(file_path, sr=16000)

    step = int(3200) #3200 sampling points every 1/5 sec
    window_frequencies = []

    for i in range(0,len(audio_data),step):
        ft = np.fft.fft(audio_data[i:i+step]) #fft returns the list N complex numbers
        freqs = librosa.fft_frequencies(sr=16000, n_fft=len(ft))
        freqs = np.fft.fftfreq(len(ft)) #fftq tells you the frequencies associated with the coefficients
        imax = np.argmax(np.abs(ft))
        freq = freqs[imax]
        freq_in_hz = abs(freq *sample_rate)
        window_frequencies.append(freq_in_hz)

    
    return window_frequencies, gender, file

In [ ]:
async def get_features(count, file):
    async with sem:
        frequencies, gender, file_name = await get_frequencies(file)

        nobs, minmax, mean, variance, skew, kurtosis =  stats.describe(frequencies)
        median   = np.median(frequencies)
        mode     = stats.mode(frequencies).mode[0]
        std      = np.std(frequencies)
        low,peak = minmax
        q75,q25  = np.percentile(frequencies, [75 ,25])
        iqr      = q75 - q25

        features_list.append([file_name, nobs, mean, skew, kurtosis, median, mode, std, low, peak, q25, q75, iqr, gender])

        print(f"\r{count}/{len(audio_files)}", end='')

        return nobs, mean, skew, kurtosis, median, mode, std, low, peak, q25, q75, iqr

In [ ]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(6000)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []


current_path = os.getcwd()
file_path = os.path.join(current_path,"CETUC", "Full")
audio_files = os.listdir(file_path)

for k, file in enumerate(audio_files):
    sent = asyncio.ensure_future(get_features(count=k+1, file=file))
    #sent = asyncio.create_task(get_features(count=k+1, files=audio_files))
    
    sents.append(sent)
 
done, _ = loop.run_until_complete(asyncio.wait(sents))

## Save to dataframe and to .csv

In [ ]:
dataframe_features = pd.DataFrame(features_list, columns = ['FileName', 'nobs', 'mean', 'skew', 'kurtosis', 'median', 'mode', 'std', 'low', 'peak', 'q25', 'q75', 'iqr', 'Gender'])
dataframe_features.to_csv('data/CETUC_Features_data.csv', index=False)

# MFCCs 

Extract MFCCs

In [32]:
async def extract_MFCCs(count, file):
    async with sem:
        file_path = os.path.join(current_path,"CETUC", "Full", file)
        
        audio_data, sample_rate =librosa.load(file_path)
        
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate)
        
        mfccs_mean = list(np.mean(mfccs.T, axis= 0))
        if file[0] == 'F':
            gender = 0
        if file[0] == 'M': 
            gender = 1
        
        sample_features = mfccs_mean
        sample_features.insert(0,str(file))
        sample_features.append(gender)
        print(f"\r{count}/{len(audio_files)}",end='')
        string = ','.join(str(item) for item in sample_features)
        async with aiofiles.open('CETUC_MFCCs_test.csv', mode='a') as f:
            await f.write(f'\n{string}')
        #features_list.append(sample_features)
        
        
        return 

In [33]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(600)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []


current_path = os.getcwd()
file_path = os.path.join(current_path,"CETUC", "Full")
audio_files = os.listdir(file_path)

for k, file in enumerate(audio_files):
    sent = asyncio.ensure_future(extract_MFCCs(count=k+1, file=file))
    
    sents.append(sent)
 
done, _ = loop.run_until_complete(asyncio.wait(sents))

20/100998

In [5]:
dataframe_features = pd.DataFrame(features_list, columns = ['FileName',
                                                            'MFCC_1',  'MFCC_2',  'MFCC_3',  'MFCC_4',  'MFCC_5',
                                                            'MFCC_6',  'MFCC_7',  'MFCC_8',  'MFCC_9',  'MFCC_10',
                                                            'MFCC_11',  'MFCC_12',  'MFCC_13',  'MFCC_14',  'MFCC_15',
                                                            'MFCC_16',  'MFCC_17',  'MFCC_18',  'MFCC_19',  'MFCC_20',
                                                            'Gender'])
dataframe_features

,FileName,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,...,MFCC_12,MFCC_13,MFCC_14,MFCC_15,MFCC_16,MFCC_17,MFCC_18,MFCC_19,MFCC_20,Gender
0,F003-0616.wav,-451.643311,120.348000,-2.872225,47.237850,-0.977582,13.857303,-4.630294,3.966301,1.860328,...,-1.193694,-4.170928,-6.661057,-0.238222,-9.994849,0.035242,-7.937756,-6.492932,-9.149591,0
1,F000-0823.wav,-309.708923,96.347252,10.261569,34.542908,6.847061,-7.365472,-4.725027,-4.484074,-1.018457,...,-4.834370,1.460012,0.723206,1.243340,3.026395,6.535556,6.572198,6.004393,2.495779,0
2,M009-0399.wav,-351.836151,133.792648,5.291138,34.379498,-3.051024,17.129290,-18.453335,10.407355,1.173181,...,-5.890615,3.383151,-4.538214,9.257086,-1.547871,3.313380,2.661654,2.677031,-0.651741,1
3,F033-0492.wav,-320.952759,113.089172,3.074278,52.792080,-5.408656,13.092081,-17.207413,4.061078,-0.398100,...,-4.014392,-2.159251,-4.752053,11.259323,-3.639835,2.808701,4.757501,1.385649,0.782601,0
4,M029-0430.wav,-369.078278,117.032066,-19.248899,41.389400,14.482154,22.141537,3.132720,-3.865579,-5.833462,...,-14.316450,-1.992915,-12.527349,-5.694921,-4.658013,-3.816001,-7.329155,-6.387514,-2.459448,1
5,F009-0753.wav,-310.293732,120.277641,-2.194679,27.556265,-8.262205,-7.890213,-18.394138,-22.403902,-8.494975,...,1.158733,-21.477530,-2.384123,5.533496,-11.425479,-4.293311,-2.286280,-9.527864,-6.867228,0
6,M030-0771.wav,-416.624542,87.730026,3.148817,32.495113,-0.724305,10.640367,-4.032876,5.090414,-6.903831,...,-4.990488,8.720636,-0.301931,2.929371,4.072585,-0.317363,2.345129,4.215478,-0.038927,1
7,M015-0405.wav,-323.869629,148.876724,19.668608,39.453651,-3.130611,9.270750,-26.070930,-6.070831,-2.409813,...,-2.438695,-0.198202,-8.019691,5.338805,-0.840717,-1.245572,-4.472536,-0.171581,-2.360906,1
8,M012-0187.wav,-513.435608,137.857819,-15.157683,42.907825,8.911463,10.530926,4.929268,1.468668,1.849850,...,1.446443,5.149837,-2.831041,7.655384,0.811686,0.335297,0.112389,-0.251837,-1.533479,1
9,M044-0172.wav,-304.906219,114.066917,-23.066444,30.073063,5.870184,-9.434823,-7.286219,-1.080048,-16.057144,...,-6.947669,-3.893003,-3.602826,1.235388,-6.142362,-2.849180,-0.917313,0.097542,-3.366011,1


In [6]:
dataframe_features.to_csv('data/CETUC_MFCCs_data.csv', index=False)

In [ ]:
current_path = os.getcwd()
file_path = os.path.join(current_path,"CETUC", "Full")
audio_files = os.listdir(file_path)
len(audio_files[50000:])

# F0

In [32]:
current_path = os.getcwd()
file_path = os.path.join(current_path,"CETUC", "Full", "F009-0399.wav")
audio_data = parselmouth.Sound(file_path)
# audio_data = audio_data.values[0]
pitch = audio_data.to_pitch()
pitch_values = pitch.selected_array['frequency']
np.mean(pitch_values)

129.92416939029846

## Extract F0

In [34]:
async def extract_F0(count, file):
    async with sem:
        file_path = os.path.join(current_path,"CETUC", "Full", file)
        audio_data = parselmouth.Sound(file_path)
        pitch = audio_data.to_pitch()
        pitch_values = pitch.selected_array['frequency']
        

        nobs_pitch, minmax_pitch, mean_pitch, variance_pitch, skew_pitch, kurtosis_pitch =  stats.describe(pitch_values)
        median_pitch   = np.median(pitch_values)
        mode_pitch     = stats.mode(pitch_values).mode[0]
        std_pitch      = np.std(pitch_values)
        low_pitch,peak_pitch = minmax_pitch
        q75_pitch,q25_pitch  = np.percentile(pitch_values, [75 ,25])
        iqr_pitch      = q75_pitch - q25_pitch

        
        if file[0] == 'F':
            gender = 0
        if file[0] == 'M': 
            gender = 1
        
        sample_features = [nobs_pitch, mean_pitch, skew_pitch, kurtosis_pitch, median_pitch, mode_pitch, std_pitch, low_pitch, peak_pitch, q25_pitch, q75_pitch, iqr_pitch]
        sample_features.insert(0,str(file))
        sample_features.append(gender)
        print(f"\r{count}/{len(audio_files)}",end='')
        string = ','.join(str(item) for item in sample_features)
        async with aiofiles.open('CETUC_F0_test.csv', mode='a') as f:
            await f.write(f'\n{string}')
        #features_list.append(sample_features)
        

        return 

In [36]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(600)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []


current_path = os.getcwd()
file_path = os.path.join(current_path,"CETUC", "Full")
audio_files = os.listdir(file_path)

for k, file in enumerate(audio_files):
    sent = asyncio.ensure_future(extract_F0(count=k+1, file=file))
    
    sents.append(sent)
 
done, _ = loop.run_until_complete(asyncio.wait(sents))

100998/100998